<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/RAG_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')
DEEPSEEK_API_KEY=userdata.get('DEEPSEEK_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU
%pip install langchain-deepseek -qU


%pip install langchain_community -qU
%pip install langchainhub -qU
%pip install langchain-text-splitters -qU

%pip install chromadb langchain-chroma -qU
%pip install langchain-chromadb -qU


# Para web scraping con LangChaig
%pip install requests -qU
%pip install beautifulsoup4 -qU



# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic
# from langchain_deepseek import ChatDeepSeek

from langchain_community.document_loaders import WebBaseLoader

import bs4


# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

bs4.SoupStrainer: Es una clase de BeautifulSoup que permite filtrar el contenido HTML durante el análisis (parsing). En lugar de analizar todo el documento HTML, solo se analizan las partes que coinciden con el filtro especificado.

class_="content_area": Este filtro indica que solo se deben analizar los elementos HTML que tengan la clase CSS content_area. El uso de class_ (con un guion bajo al final) es necesario porque class es una palabra reservada en Python.

In [ ]:
import os

# Define un USER_AGENT válido
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"


bs4_strainer = bs4.SoupStrainer(id="bodyContent")
loader = WebBaseLoader(
    web_paths=("https://es.wikipedia.org/wiki/Edificio_Chrysler",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()


docs

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

all_splits

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

embeddings

In [ ]:
from langchain_community.vectorstores import Chroma

# Crear vector store
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings
)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})





In [ ]:
pregunta = "Cuantas ventanas tiene el edificio Chrysler ?"
retrieved_docs = retriever.invoke(pregunta)

for doc in retrieved_docs:
    display(Markdown(doc.page_content))
    print("---\n")


In [ ]:
contexto = ' '.join([doc.page_content for doc in retrieved_docs])

contexto

In [ ]:
# Modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=0.7)




In [ ]:
respuesta = llm.invoke(f"""Usando el contexto proporcionado, responde a la pregunta. Si no lo sabes, responde simplemente 'No lo se'
           Pregunta: {pregunta}.
           Contexto: {contexto}
""")

display(Markdown(respuesta.content))